In [ ]:
import warnings

import pandas as pd
import torch
import nltk

from tqdm import tqdm
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW, get_scheduler
import torch.nn as nn
from torch.utils.data import Dataset

warnings.filterwarnings('ignore')

In [ ]:
# This refers to `b. Datasets\XYZ Store - Item Reviews`
train_data = pd.read_csv('/kaggle/input/nlp-cleaneddataset/Train Cleaned Data.csv')
test_data = pd.read_csv('/kaggle/input/nlp-cleaneddataset/Test Cleaned Data.csv')

In [3]:
# train_data = pd.read_csv('/kaggle/input/nlphackathondata/Train Data.csv')
# test_data = pd.read_csv('/kaggle/input/nlphackathondata/Test Data.csv')

In [4]:
# train_data = pd.read_csv("/kaggle/input/nlp-hackathon-smallsubset/Train Data - Very Small.csv")
# test_data = pd.read_csv("/kaggle/input/nlp-hackathon-smallsubset/Test Data - Very Small.csv")

In [5]:
# train_data.fillna(" ", inplace=True)
# test_data.fillna(" ", inplace=True)

In [6]:
nltk.data.path.append('/kaggle/input/nltk-stopwrods/stopwords')

# # Test if NLTK can access stopwords
from nltk.corpus import stopwords
print(stopwords.words('english')[:10]) 

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [7]:
nltk.data.path.append('/kaggle/input/nltk-wordnet')

# Test if wordnet is working
from nltk.corpus import wordnet
print(wordnet.synsets('example'))

[Synset('example.n.01'), Synset('model.n.07'), Synset('exemplar.n.01'), Synset('example.n.04'), Synset('case.n.01'), Synset('exercise.n.04')]


In [8]:
# # nltk.download('stopwords')
# # nltk.download('wordnet')

# import re
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer, PorterStemmer
# from nltk.tokenize import word_tokenize
# import spacy

# # Initialisations
# stop_words = set(stopwords.words('english'))
# lemmatizer = WordNetLemmatizer()

# stemmer = PorterStemmer()
# nlp = spacy.load("en_core_web_sm")

# def preprocess_text(text):
#     """
#     Enhanced function to clean and preprocess text data.
#     """
#     # Convert to lowercase
#     text = text.lower()

#     # Remove special characters, numbers, and punctuation
#     text = re.sub(r'[^a-z\s]', '', text)
#     tokens = word_tokenize(text)

#     # Handle negations (e.g., "not good" -> "not_good")
#     tokens = ['not_' + tokens[i + 1] if tokens[i] == 'not' and i + 1 < len(tokens) else tokens[i] for i in range(len(tokens))]

#     # Remove stopwords
#     tokens = [word for word in tokens if word not in stop_words]

#     # Perform lemmatisation
#     tokens = [lemmatizer.lemmatize(word) for word in tokens]

#     # Optionally perform stemming (comment out if not needed)
#     tokens = [stemmer.stem(word) for word in tokens]

#     # Remove short tokens (e.g., "a", "i")
#     tokens = [word for word in tokens if len(word) > 1]

#     # Handle domain-specific stopwords (optional, example list)
#     domain_specific_stopwords = {'example'}
#     tokens = [word for word in tokens if word not in domain_specific_stopwords]

#     # Use Named Entity Recognition (NER) to remove entities if needed
#     text = " ".join(tokens)
#     doc = nlp(text)
#     tokens = [token.text for token in doc if not token.ent_type_]

#     # Join tokens back to text
#     return " ".join(tokens)

# # Apply preprocessing to training and test data
# train_data['Cleaned_Review'] = train_data['Review'].apply(preprocess_text)
# test_data['Cleaned_Review'] = test_data['Review'].apply(preprocess_text)

In [9]:
# train_data.to_csv("Train Cleaned Data.csv", index=False)
# test_data.to_csv("Test Cleaned Data.csv", index=False)

In [10]:
# display(train_data.head(2))
# display(train_data.tail(2))
# display(test_data.head(2))
# display(test_data.tail(2))

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
class ReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, labels=None, max_len=256):
        """
        Args:
            reviews: List of review texts.
            tokenizer: BERT tokenizer instance.
            labels: Optional, list of target labels.
            max_len: Maximum length of tokenized sequences.
        """
        self.reviews = reviews
        self.labels = labels  # Labels can be None for test data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        # Retrieve a single review and convert it to string
        review = str(self.reviews[item])

        # Tokenize the review
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Prepare dictionary for model input
        data = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
        }

        # Add label only for training/validation data
        if self.labels is not None:
            data['labels'] = torch.tensor(self.labels[item], dtype=torch.long)

        return data

In [13]:
# Combine 'Cleaned_Review' and 'Review_Title' into a single text column
train_data_review_values = train_data['Review_Title'] + " " + train_data['Cleaned_Review']
test_data_review_values = test_data['Review_Title'] + " " + test_data['Cleaned_Review']

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Prepare Train Dataset
train_dataset = ReviewDataset(
    reviews=train_data_review_values.values,  # Pass cleaned reviews
    labels=train_data['Rating'].values,  # Pass labels
    tokenizer=tokenizer,
    max_len=256
)

# Create Test Dataset (No labels)
test_dataset = ReviewDataset(
    reviews=test_data_review_values.values,
    tokenizer=tokenizer
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [15]:
class ReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, labels=None, max_len=256):
        """
        Args:
            reviews: List of review texts.
            tokenizer: BERT tokenizer instance.
            labels: Optional, list of target labels (for training).
            max_len: Maximum length of tokenized sequences.
        """
        self.reviews = reviews
        self.labels = labels  # Labels can be None for test data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        # Retrieve a single review and convert it to string
        review = str(self.reviews[item])

        # Tokenize the review
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Prepare dictionary for model input
        data = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
        }

        # Add label if it exists (for training only)
        if self.labels is not None:
            data['labels'] = torch.tensor(self.labels[item], dtype=torch.long)

        return data

In [16]:
# Define hyperparameters to test
# learning_rates = [5e-5, 3e-5, 2e-5]
learning_rates = [3e-5]
batch_sizes = [32]
epochs = 4

best_accuracy = 0
best_params = {}

# Grid search for hyperparameters
for lr in learning_rates:
    for batch_size in batch_sizes:
        print(f"Testing: LR={lr}, Batch Size={batch_size}")

        # Re-initialize dataset and dataloaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

        # Re-initialize model and optimizer
        model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
        model.load_state_dict(torch.load("/kaggle/input/nlp_bestmodel_99.93/pytorch/default/1/best_model.pt"))
        model = model.to(device)
        optimizer = AdamW(model.parameters(), lr=lr)

        # Learning rate scheduler
        scheduler = get_scheduler('linear', optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * epochs)

        # Training loop
        for epoch in range(epochs):
            model.train()
            for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs} Training", unit="batch"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                optimizer.zero_grad()
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                loss.backward()

                # Gradient clipping
                nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

                optimizer.step()
                scheduler.step()

        # Evaluate model performance
        model.eval()
        predictions, true_labels = [], []
        with torch.no_grad():
            for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs} Training", unit="batch"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, dim=1)
                predictions.extend(preds.cpu().numpy())
                true_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(true_labels, predictions)
        print(f"Accuracy for LR={lr}, Batch Size={batch_size}: {accuracy}")

        # Save best parameters
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'learning_rate': lr, 'batch_size': batch_size}
            # Save the model
            torch.save(model.state_dict(), "best_model.pt")

print("Best Hyperparameters:", best_params)
print("Best Accuracy:", best_accuracy)

Testing: LR=3e-05, Batch Size=32


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 4/4 Training: 100%|██████████| 1400/1400 [05:38<00:00,  4.14batch/s]


Accuracy for LR=3e-05, Batch Size=32: 0.9926782445644895
Best Hyperparameters: {'learning_rate': 3e-05, 'batch_size': 32}
Best Accuracy: 0.9926782445644895


In [17]:
# Load the best hyperparameters
best_lr = best_params['learning_rate']
best_batch_size = best_params['batch_size']

# Create DataLoader for test dataset
test_loader = DataLoader(test_dataset, batch_size=best_batch_size, shuffle=False)

# Reinitialise model and load the saved weights
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.load_state_dict(torch.load("best_model.pt"))  # Load saved weights
model = model.to(device)

# Set the model to evaluation mode
model.eval()

# Make predictions on test data
predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)
        predictions.extend(preds.cpu().numpy())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
best_batch_size

32

In [19]:
df = pd.DataFrame({
    'ID': test_data['ID'],
    'Rating': predictions
    })

display(df.head(2))
display(df.tail(2))

,ID,Rating
0,0,1
1,1,1


,ID,Rating
14930,16373,1
14931,16374,1


In [20]:
df.to_csv('NLP_Hackathon_GC_Mini_2.csv', index=False)